In [1]:
import pandas as pd
import pickle
from datetime import datetime


In [2]:
original_path = '../../data/april2022/original'
processed_path = '../../data/april2022/processed'


In [3]:
calendar_path = f'{original_path}/calendar.txt'
routes_path = f'{original_path}/routes.txt'
stop_times_path = f'{original_path}/stop_times.txt'
stops_path = f'{original_path}/stops.txt'
trips_path = f'{original_path}/trips.txt'


In [4]:
calendar = pd.read_csv(calendar_path, delimiter=',', header=0)

routes = pd.read_csv(routes_path, delimiter=',', header=0)

stop_times = pd.read_csv(stop_times_path, delimiter=',', header=0)
stop_times = stop_times.astype({'stop_id': str})

stops = pd.read_csv(stops_path, delimiter=',', header=0)
stops = stops.astype({'stop_id': str})

trips = pd.read_csv(trips_path, delimiter=',', header=0)


In [5]:
joined = pd.merge(routes, trips, on='route_id', how='inner')
joined = pd.merge(joined, stop_times, on='trip_id', how='inner')
joined = pd.merge(joined, stops, on='stop_id', how='inner')
joined = joined.sort_values(
    by=['route_id', 'trip_short_name', 'stop_sequence'], ascending=True)


In [6]:
tratte = joined[[
    "route_id",
    "route_long_name",
    "arrival_time",
    "departure_time",
    "trip_short_name",
    "stop_id",
    "stop_sequence",
    "stop_name"
]]


def trycast_datetime(time_str):
    try:
        datetime.strptime(time_str, "%H:%M:%S")
    except:
        return False
    return True


arrival_time_criterion = tratte['arrival_time'].map(trycast_datetime)
departure_time_criterion = tratte['departure_time'].map(trycast_datetime)

tratte = tratte[arrival_time_criterion & departure_time_criterion]
tratte = tratte.reset_index(drop=True)


In [7]:
weeks = pd.merge(trips, stop_times, on='trip_id', how='inner')
weeks = pd.merge(weeks, calendar, on='service_id', how='inner')


In [8]:
weeks['times_in_week'] = weeks['monday'] + weeks['tuesday'] + weeks['wednesday'] + \
    weeks['thursday'] + weeks['friday'] + weeks['saturday'] + weeks['sunday']

weeks = weeks[['route_id', 'trip_short_name',
               'stop_id', 'stop_sequence', 'times_in_week']]

weeks = weeks.sort_values(
    by=['trip_short_name', 'stop_sequence'], ascending=True)
weeks = weeks.drop_duplicates()
weeks = weeks.reset_index(drop=True)


In [9]:
with open(f"{processed_path}/trenord.pickle", 'wb') as f:
    pickle.dump([calendar, routes, stop_times, stops, trips, tratte, weeks], f)
